# Image Classificaiton 

In the notebook we will walkthrough an end-to-end image classification. The objective is to take a set of images and classify them into different categoreis based on object oimages. The categories used are as follows:
* Landscapes
* City Scapes
* Food
* Concerts
* Group Photos
* My Face


Each category will have around 100 images which we will developed a training model to learn various classification. 

We will be using the Convolutional Neural Network for the classification. 

Big thanks to [DarkBones](https://github.com/DarkBones/CNN-Image-Classifier/) the datasets and source code. Here is the [narative](https://github.com/DarkBones/CNN-Image-Classifier/blob/master/Capstone%20Proposal.pdf) by DarkBones. 

Without futher a-do, lets get started!

# Step 1: Image Processing


### Imports

In [1]:
import os
import numpy as np
from glob import glob
from sklearn.datasets import load_files
from keras.utils import np_utils
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, load_img
import re
import random
import shutil

/home/redne/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 1: Initialize Parameters

In [2]:
root_dir = os.path.join('..', 'application', 'images' )
originals_dir = os.path.join(root_dir, 'original')
training_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'validation')
test_dir = os.path.join(root_dir, 'test')

target_imagesize = (256, 256)

# size of the test and validation sets as compared to the total amount of images
test_size = 0.2
validation_size = 0.2

clear_existing_data = False 
# if true, data in training, test and validation directories will be deleted before splitting the 
# .. the data in the original direcories

random_seed = 7

# the amount of images in the training, validation and test sets
training_count = 0
validation_count = 0
test_count = 0

# list of categories
categories = []

###  Step 1: Split the dataset

A model will be training with the training set and validated iwh the validation set. __Whenever the perfomrance on the validation set improves the weights oof the model are saved__ _(overwriting the previously best performing weights)_. Several models will be training in this fassion. 

AFter all models have been trainined, we choose the one that performs the best on data it hasnt seen before: __the test set.__

This is why, in order to train our model and test how well the model is performing, the dataset must be split up randomly into ___ training, validation and test sets__.

The `__split_dataset` funciton will go through the images in the original dataset and split them into training, validation and test sets according to the test_size and validation_size parameters set above. 

Both functions can only be called from inside the class, since the plublic `initalize()` function will call them.

In [3]:
# function to remove all files in given directory 
def __empty_directory(path):
    for file in os.listdir(path):
        os.remove(os.path.join(path, file))
    return

In [4]:
# splits images in original directory into training, test and validation directories
def __split_dataset():
    random.seed(random_seed)
    
    size_count = 0
    for category in os.listdir(originals_dir):
        # make a new directory where they don't exist and empty existing directories
        for p in [re.split(r'[\\/]', training_dir)[-1], re.split(r'[\\/]', val_dir)[-1], re.split(r'[\\/]', test_dir)[-1]]:
            if not os.path.exists(os.path.join(root_dir, p, category)):
                os.makedirs(os.path.join(root_dir, p, category))
            if clear_existing_data == True:
                __empty_directory(os.path.join(root_dir, p, category))
            
        # collect all the files in the originals directory
        files = []
        for file in os.listdir(os.path.join(originals_dir, category)):
            files.append(file)
        
        # calculate the training, validation and test set sizes
        test_count = round(len(files) * test_size)
        validation_count = round(len(files) * val_size)
        train_count = len(files) - test_count - validation_count
        
        # randomly shuffle the array of files
        random.shuffle(files)
        
        for i, file in enumerate(files):
            location = None
            if i < test_count:
                location = test_dir
            elif i < test_count + validation_count:
                location = val_dir
            else:
                location = training_dir
                
            shutil.copyfile(os.path.join(originals_dir, category, file), os.path.join(location, category, file))
    return

##  2: Getting additional class parameters

`ImagePreprpcessor` class described in this notebook may need to know additional information with regards to how large each dataset is and what the names of the categories are. 

The `training_count`, `validation_count`, `test_count`, and `categories` variables are not assigned any values by default. After the data has been split, we can count how many images are in each of the datasets and what categories are used.

In [5]:
# returns an array with the category names
def __get_categories():
    return [item[len(originals_dir)+1:] for item in sorted(glob(os.path.join(originals_dir, "*")))]

# returns the sizes of the training, validation and test sets
def __get_dataset_sizes():
    train_size = sum([len(files) for r, d, files in os.walk(training_dir)])
    validation_size = sum([len(files) for r, d, files in os.walk(val_dir)])
    test_size = sum([len(files) for r, d, files in os.walk(test_dir)])
    
    return train_size, validation_size, test_size

##  3: Initialize the class
Now that all private functions are in place, we can call the public master function to call them in the correct order.

The `intialize` function is the only funcitno that needs to be called after inailizing the `ImagePreProcessor` class. It will preprocess to split the datasets and the class will store information about the datasets. 

In [6]:
def initialize(self):
    __split_dataset()
    
    training_count, validation_count, test_count = __get_dataset_sizes()
    categories = __get_categories()

## 4: Convert images into tensors

The code sectio below converts these images into ___tensors;___ matrices of numeric values representing how bright each pixel in the image is. The numeric values are then normalized so they are all within a range of between 0 and 1, rather than between 0 and 255. 

Then __normaliztion__ makes it easier for the model to train, as all pixels are now in the same range relative to the brightest pixel in each particular image. The brightest pixel in a dark image will still be of value 1, even though it may not have been 255 before the normalization. 

In [7]:
# take a list of image filepaths and retunr a lost of 4D tensors
def file_to_tensor(self, file):
    img = image.load_img(file, target_size=target_imagesize)
    x = image.img_to_array(img)
    x *= (1.0/x.max()) # set the range of the tensor values between 0 and 1
    return np.expand_dims(x, axis=0)

def files_to_tensors(self, files):
    list_of_tensors = [file_to_tensor(file) for file in files]
    return np.vstack(list_of_tensors)

## 5: Load datasets
The function below takes a directory and returns a list of image locations, along with the list of one-hot encoded targets

In [8]:
# load file locations and labels
def load_dataset(path):
    data = load_fileS(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), max(data['terget'])+1)
    return files, targets

# Step 2 - Choosing a Model

This is a continuaiton from the pervious notebook __1_image_preprocessing.__ In This section we will slect how different models are trained and how we can determine which of the models achieved the best results. 

The models we will training include:
* neuralnet_model
* cnn_model
* VGG19
* ResNet50
* InceptionV3

All modes that are trained will be saved in a list. AFter training we will evaluate the performance of the modes from the persepctive for the `F1 Score`. Thereafter, we use the testing set (which the model shavent seen before) to calculate their F1 Score. 

___Key Objective: The model with the hightest F1 score on the testing set, is the model we're going to use.___ 


Big thanks to [DarkBones](https://github.com/DarkBones/CNN-Image-Classifier/) the datasets and source code. Here is the [narative](https://github.com/DarkBones/CNN-Image-Classifier/blob/master/Capstone%20Proposal.pdf) by DarkBones. 


### Import Dependencies

In [3]:
from image_preprocessor import ImagePreprocessor
## import models
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3

## kares key imports
import keras.callbacks as callbacks
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import load_model
from keras.callbacks import TensorBoard

import re
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

### Set Parameters

In [4]:
root_dir = os.path.join('..', 'application', 'images')
originals_dir = os.path.join(root_dir, "original")
training_dir = os.path.join(root_dir, "train")
test_dir = os.path.join(root_dir, "test")
val_dir = os.path.join(root_dir, "validation")

target_imagesize = (256, 256)

clear_existing_data = True # if true, data in training, test and validation directories will be deleted before splitting the data in the originals directory
augment_data = True # whether images should be augmented during preprocessing
augmentations = 20 # how many augmentations to make for each original image

random_seed = 7

epochs = 10
batch_size = 50
saved_models_dir = os.path.join('..', 'application', 'saved_models')

## Initiate preprocess

In [5]:
preprocessor = ImagePreprocessor()
preprocessor.root_dir = root_dir
preprocessor.originals_dir = originals_dir
preprocessor.training_dir = training_dir
preprocessor.test_dir = test_dir
preprocessor.val_dir = val_dir
preprocessor.random_seed = random_seed
preprocessor.target_imagesize = target_imagesize
preprocessor.clear_existing_data = clear_existing_data

preprocessor.initialize()
categories = preprocessor.categories
training_count = preprocessor.training_count
validation_count = preprocessor.validation_count
test_count = preprocessor.test_count

6 image categories
464 total images

278 training images
93 validation images
93 test images

Categories:
  - animal
  - city_scape
  - food
  - group
  - landscape
  - me


In [5]:
__get_categories()

NameError: name '__get_categories' is not defined

###  Generate augmented images

bc our dataset is relatively small (only a few hundred images), we will artificailly generate more data by augmenting the images. The original images are randomly rotated, zoomed and / or flipped horizontally. 

In [6]:
img_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255,
    fill_mode='reflect')

train_generator = img_datagen.flow_from_directory(training_dir,
                                                   target_size=target_imagesize,
                                                   batch_size=augmentations,
                                                   shuffle=True,
                                                   seed=random_seed)

validation_generator = img_datagen.flow_from_directory(val_dir,
                                                   target_size=target_imagesize,
                                                   batch_size=augmentations,
                                                   shuffle=True,
                                                   seed=random_seed)

Found 278 images belonging to 7 classes.
Found 93 images belonging to 7 classes.


### Benchmark model: Random guessing

To get a better understanding of how well our model work, we will first create a model that just randomly guesses the categories and calculates its F1 score. 

In [7]:
def random_guesses(count):
    guesses = []
    for i in range(count):
        guesses.append(random.randint(0, len(categories)))
    return guesses

### Calculating the F1 score

To measure how good the model is at predicting the correct category for images, we calculate the model's F1 score. The higher this score, the better the model is performing. IN the ned, we choose the model that has the highest F1 score on the images in the test set. 

In [8]:
def predict_category(img_path, pred_model):
    img_tensor = preprocessor.file_to_tensor(img_path)
    try:
        h = pred_model.predict(img_tensor)
    except:
        img_tensor = img_tensor.reshape(1,x_length)
        h = pred_model.predict(img_tensor)
    return categories[np.argmax(h)]

In [9]:
def f1_score_cal(model=None):
    test_images = np.array(glob(os.path.join(test_dir, "*", "*")))
    y_true = []
    y_pred = []
    for img in test_images:
        y_true.append(categories.index(re.split(r'[\\/]',img)[-2]))
        if model != None:
            pred = predict_category(img, model)
            y_pred.append(categories.index(pred))
        
    if model == None:
        y_pred = random_guesses(len(y_true))
    
    return f1_score(y_true, y_pred, average='weighted')

## Training

We will store the training models in a list called `models`, along with another list of the model names so we can erfer to them later. After all modesl are trained, we will calculate their `f1 scores` and store them in the `f1_scores` list. 

In [10]:
models = []
modelnames = []
f1_scores = []

models.append(None)
modelnames.append("random")

### Benchmark model: Traditional Neural Network

To test the effectiveness of Convolutional Neural Networks, we will also create a `traditional Feed-Froward Neural Network` to see how it compares against `CNN`.

In [11]:
# load the file locations and their labels 'y'
train_files, y_train = preprocessor.load_dataset(training_dir)
val_files, y_val = preprocessor.load_dataset(val_dir)
test_files, y_test = preprocessor.load_dataset(test_dir)

# load training, validation, and test matrices
x_train = preprocessor.files_to_tensors(train_files)
x_val = preprocessor.files_to_tensors(val_files)
x_test = preprocessor.files_to_tensors(test_files)

# calculate how many pixels are in the images
x_length = 1
for n in x_train.shape[1:]:
    x_length *= n

# reshape the tensors into single dimensions
x_train = x_train.reshape(len(x_train),x_length)
x_val = x_val.reshape(len(x_val),x_length)
x_test = x_test.reshape(len(x_test),x_length)

/home/redne/anaconda3/lib/python3.6/site-packages/PIL/Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [12]:
neuralnet_model = Sequential()
neuralnet_model.add(Dense(512, input_dim=x_length, activation='relu'))
neuralnet_model.add(Dense(512, activation='relu'))
neuralnet_model.add(Dense(512, activation='relu'))
neuralnet_model.add(Dropout(0.3))
neuralnet_model.add(Dense(len(categories)+1, activation='softmax'))
neuralnet_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               100663808 
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 3591      
Total params: 101,192,711
Trainable params: 101,192,711
Non-trainable params: 0
_________________________________________________________________


In [13]:
neuralnet_model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath=os.path.join(saved_models_dir, 'traditional_neuralnet.hdf5'), 
                           verbose=1, save_best_only=True)

neuralnet_model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=(x_val, y_val),
                    verbose=1,
                    callbacks=[checkpointer])

neuralnet_model.load_weights(filepath=os.path.join(saved_models_dir,'traditional_neuralnet.hdf5'))
models.append(neuralnet_model)
modelnames.append("benchmark")

Train on 278 samples, validate on 93 samples
Epoch 1/10
278/278 [==============================] - 6s 23ms/step - loss: 2.0038 - acc: 0.1511 - val_loss: 1.7580 - val_acc: 0.3656

Epoch 00001: val_loss improved from inf to 1.75800, saving model to ../application/saved_models/traditional_neuralnet.hdf5
Epoch 2/10
278/278 [==============================] - 5s 18ms/step - loss: 1.7455 - acc: 0.3058 - val_loss: 1.6488 - val_acc: 0.2796

Epoch 00002: val_loss improved from 1.75800 to 1.64878, saving model to ../application/saved_models/traditional_neuralnet.hdf5
Epoch 3/10
278/278 [==============================] - 5s 18ms/step - loss: 1.6515 - acc: 0.3489 - val_loss: 1.5880 - val_acc: 0.3978

Epoch 00003: val_loss improved from 1.64878 to 1.58802, saving model to ../application/saved_models/traditional_neuralnet.hdf5
Epoch 4/10
278/278 [==============================] - 5s 18ms/step - loss: 1.5594 - acc: 0.3957 - val_loss: 1.5216 - val_acc: 0.3978

Epoch 00004: val_loss improved from 1.5880

## Construct a new CNN Model from scratch

in teh code below, we will create a new Convolutional Neural Network from scratch.

In [42]:
cnn_model = Sequential()
cnn_model.add(Conv2D(filters=16,
                     kernel_size=2, 
                     strides=(1, 1), 
                     padding='same', 
                     activation='relu', 
                     input_shape=(target_imagesize[0], target_imagesize[1], 3)))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=2))
cnn_model.add(Dropout(0.3))
cnn_model.add(Flatten())
cnn_model.add(Dense(512, activation='relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(512, activation='relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(len(categories)+1, activation='softmax'))

cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 256, 256, 16)      208       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 128, 128, 32)      2080      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 64, 64, 64)        8256      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 32, 32, 128)       32896     
__________

In [44]:
checkpointer = ModelCheckpoint(filepath=os.path.join(saved_models_dir,'cnn_from_scratch.hdf5'), 
                           verbose=1, save_best_only=True)

cnn_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=['accuracy'])
cnn_model.fit_generator(train_generator,
                        steps_per_epoch=training_count, 
                        #epochs=epochs,
                        epochs = 3,
                        validation_data = validation_generator,
                        validation_steps=validation_count,
                        callbacks=[checkpointer],
                        verbose=1)

cnn_model.load_weights(filepath=os.path.join(saved_models_dir,'cnn_from_scratch.hdf5'))
models.append(cnn_model)
modelnames.append("new cnn")

Epoch 1/3
  7/278 [..............................] - ETA: 8:12 - loss: 1.7971 - acc: 0.2500

/home/redne/anaconda3/lib/python3.6/site-packages/PIL/Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


278/278 [==============================] - 577s 2s/step - loss: 1.7717 - acc: 0.2552 - val_loss: 1.7288 - val_acc: 0.2924

Epoch 00001: val_loss improved from inf to 1.72880, saving model to ../application/saved_models/cnn_from_scratch.hdf5
Epoch 2/3
278/278 [==============================] - 579s 2s/step - loss: 1.7465 - acc: 0.2676 - val_loss: 1.7056 - val_acc: 0.3276

Epoch 00002: val_loss improved from 1.72880 to 1.70563, saving model to ../application/saved_models/cnn_from_scratch.hdf5
Epoch 3/3
278/278 [==============================] - 575s 2s/step - loss: 1.7267 - acc: 0.2829 - val_loss: 1.6811 - val_acc: 0.3287

Epoch 00003: val_loss improved from 1.70563 to 1.68106, saving model to ../application/saved_models/cnn_from_scratch.hdf5


## Transfer Learning

__Transfer Learning__ is taking an existing, pre trained model and stripping off the final layer and replacing it with our won layers so it can classify the objects we'er looking for. We will use three different pre-trained netwokrs to see how they compare:
* VGG19
* ResNet50
* InceptionV3

In [ ]:
base_models = []
base_models.append(VGG19(include_top=False, 
                        weights = 'imagenet',
                        input_shape = (target_imagesize[0], target_imagesize[1], 3)))
base_models.append(ResNet50(include_top=False,
                           weights = 'imagenet',
                           input_shape = (target_imagesize[0], target_imagesize[1], 3)))
base_models.append(InceptionV3(include_top=False,
                              weights = 'imagenet', 
                              input_shape = (target_imagesize[0], target_imagesize[1], 3)))

base_modelnames = "VGG19 ResNet50 InceptionV3".split()

## Train pre-trained models

In [ ]:
#train the models
for i, model in enumerate(base_models):
    print("")
    print("Training Models: %s" % base_modelnames[1])
    
    checkpointer = ModelCheckpoint(filepath=os.path.join(saved_models_dir,
                                                        base_modelnames[i] + '.hdf5'))
    
    for layer in model.layers:
        layer.trainable = False
        
    # custom layers
    cus_layers = model.output
    cus_layers = Flatten()(cus_layers)
    cus_layers = Dense(1024, activation='relu')(cus_layers)
    cus_layers = Dropout(0.3)(cus_layers)
    cus_layers = Dense(512, activation='relu')(cus_layers)
    cus_layers = Dropout(0.3)(cus_layers)
    cus_layers = Dense(len(categories)+1, activation='relu')(cus_layers)
    predictions = Dense(len(categories)+1, activation='softmax')(cus_layers)
    
    # create the final model
    model_final = Model(inputs = model.input, outputs = predictions)
    
    # compile the model
    model_final.compile(loss='categorical_crossentropy', optimizer = optimizers.SGD(lr=0.0001,
                                                                                   momentum=0.9),
                       metrics=['accuracy'])
    
    # train the model
    model_final.fit_generator(train_generator, 
                             steps_per_epoch = training_count // 10, 
                             epochs = 3, 
                             validation_data = validation_generator, 
                             validation_steps = validation_count // 10,
                             callbacks=[checkpointer],
                             verbose=1)
    
    model_final.load_weights(finalpath=os.path.join(saved_models_dir, base_modelnames[i] + '.hdf5'))
    
    models.append(model_final)
    modelnames.append(base_modelnames[i])


Training Models: ResNet50
Epoch 1/3


/home/redne/anaconda3/lib/python3.6/site-packages/PIL/Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


 2/27 [=>............................] - ETA: 8:29 - loss: 2.1483 - acc: 0.0806

## Choosing the model
Now that all our models ar etrained, we can calculate their F1 scores on the test set. Important is that none of the models have seen data from the test set during the training phase, so it will be good indicaiton of how th emodel will perform in a real-world scenarion

In [ ]:
y_pred = []

# load the file locations and their labels 'y'
test_files, y_test = preprocessor.load_dataset(test_dir)

# load training, validation, and test matrices
x_test = preprocessor.files_to_tensors(test_files)

In [ ]:
for i, model in enumerate(models):
    f1score = f1_score_cal(model)
    f1_scores.append(f1score)
    print(modelnames[i], f1score)

In [ ]:
fig, ax = plt.subplots()

index = np.arange(6)

rects1 = ax.bar(index, f1_scores)

ax.set_ylabel('F1 Score')
ax.set_title('F1 Scores per model')
ax.set_xticks(index)
ax.set_xticklabels(modelnames)

fig.tight_layout()
plt.show()

## Conclusion
Knowing that a higher F1 score is better, we can see that the `InceptionV3` model is outperforming all the other models with an `F1 score of 0.74`. We will take the InceptionV3 model and fine tune in the next section. 